In [1]:
import numpy as np
from sklearn.neighbors import KernelDensity
from itertools import product
import scipy.stats as st

D:\anaconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# load the links calculated from ES_calculation
# links has 3 columns: 
#   the first column is an integer between 0 to 24 refer to one of the 25 points in SCA;
#   the second column is an integer between 0 to 575999 refer to one point around the globe linked to SCA
#   the third column is the number of ES between the two points
links = np.load('SCA_links_corrected2.npy')

In [8]:
# load the null model of KDE
'''null_model = np.load('null_model.npy')
mean = np.mean(null_model, axis = 0)
std = np.std(null_model, axis = 0)
perc90 = st.scoreatpercentile(null_model, 90, axis = 0)
perc95 = st.scoreatpercentile(null_model, 95, axis = 0)
perc99 = st.scoreatpercentile(null_model, 99, axis = 0)
perc995 = st.scoreatpercentile(null_model, 99.5, axis = 0)
perc999 = st.scoreatpercentile(null_model, 99.9, axis = 0)'''
perc999 = np.load('KDE_null_sig0001.npy')

In [4]:
# randomly select 10000 links
selector = np.zeros(links.shape[0], dtype = 'bool')
selector[:10000] = 1
selector = np.random.permutation(selector)
links_selected = links[:,1][selector]

In [5]:
# generate the grids
lat = np.arange(-50,50,0.25)
lon = np.arange(-180,180,0.25)
coords = np.array(list(product(lat, lon)))
X, Y = np.meshgrid(lon, lat)
xy = np.vstack([Y.ravel(), X.ravel()]).T
xy *= np.pi / 180.

In [6]:

values = np.vstack([coords[links_selected, 0], coords[links_selected, 1]]).T
values *= np.pi/180
bw_opt = .2 * values.shape[0]**(-1./(2+4))

In [7]:
# compute sample KDE
kde = KernelDensity(bandwidth=bw_opt, metric='haversine', kernel='gaussian', algorithm='ball_tree')
kde.fit(values)
dat = np.exp(kde.score_samples(xy))
dat = dat.reshape(X.shape)

In [18]:
# sig_region is an array of locations at which KDE is significantly high
sigdat = (dat > perc999)
sigdat = sigdat.flatten()
sig_region = np.where(sigdat)[0]

In [23]:
# sig_links is the set of significant links to SCA, a subset of links
sig_link_row = np.where(np.in1d(links[:,1], sig_region))
sig_links = links[sig_link_row]

In [25]:
np.save('SCA_sig_links.npy',sig_links)

In [27]:
np.save('KDE_null_sig0001',perc999)